# StyleGan2-ADA Training

Made by [Artem Konevskikh](http://aiculedssul.net/)

Based on [dvschultz notebooks](https://github.com/dvschultz/ml-art-colabs)

In this notebook we will do transfer learning with StyleGAN2 and custom datasets.

This means we will not train GAN on our images from scratch (as it takes about two weeks) but we will use the model already trained on the other images as a starting point. It will reduce training time to about 10 hours by skipping first stages where neural network learns low level features of images that are almost the same for any kind of images.

In [ ]:
#@title GPU Check
#@markdown You can check which GPU you got. V100 is perfect, P100 and T4 is good, but if you got K80 its better to restart the notebook by selecting **Disconnect and delete runtime** in **Runtime** menu, because it is very slow
# %tensorflow_version 1.x
!nvidia-smi -L


In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to load pretrained models and to save the results.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Libraries
#@markdown At some point it will show you Restart Runtime button. Please restart it by pressing this button
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!bash ./Miniconda3-py39_23.3.1-0-Linux-x86_64.sh -b -u -p /usr/local 
import os
if os.path.isdir('/content/anaconda'): root_path = '/content/anaconda'
else: root_path = '/usr/local/'
os.environ['PATH'] += f":{root_path}/bin"
!conda install python=3.9
!pip install setuptools==59.5.0 ninja opensimplex torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install click psutil scipy

In [ ]:
#@title Install Stylegan
#@markdown StyleGAN2-ada will be installed to your Google Drive to speed up the training process

#@markdown Run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

import os
import shlex
if os.path.isdir("/content/drive/MyDrive/stylegan2ada"):
    %cd "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir stylegan2ada
    %cd stylegan2ada
    !git clone https://github.com/artem-konevskikh/stylegan2-ada-pytorch.git
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/artem-konevskikh/stylegan2-ada-pytorch.git
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    # %cd pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../




In [ ]:
#@title Data Preparation
#@markdown Input image directory
input_dir = '/content/drive/MyDrive/scraped-images/Mycelium-dataset-512' #@param {type: "string"}
#@markdown Path to the zip file where converted dataset will be stored
dataset_file = '/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/datasets/mycelium.zip' #@param {type: "string"}

if not dataset_file.endswith('.zip'):
  dataset_file += '.zip'
input_dir = shlex.quote(input_dir)
dataset_file = shlex.quote(dataset_file)
!python dataset_tool.py --source {input_dir} --dest {dataset_file}

In [ ]:
#@title Train a custom model

#@markdown Path to the dataset zip file
dataset = "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/datasets/mycelium.zip" #@param {type: "string"}

#@markdown For transfer learning set it to `ffhq256`, `ffhq512` or `ffhq1024`accordingly to your images resolution.<br />
#@markdown If you want to resume training process, provide the path to your latest .pkl file
resume_from = "ffhq512" #@param {type: "string"}

dataset = shlex.quote(dataset)
resume_from = shlex.quote(resume_from)
#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap=1 --cfg='11gb-gpu' --data={dataset} --aug=noaug --mirror=False --mirrory=False --metrics=None --resume={resume_from}

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. 

Every ~40min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder (`/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results`) in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.